In [1]:
import pyspark
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from math import radians, cos, sin, asin, sqrt


In [2]:
account_name = "REDACTED"
account_key = "REDACTED"

spark = (
    SparkSession
        .builder
        .master('local[*]')
        .appName("Ingestion")
        .config("spark.driver.memory", "4g")
        .config("fs.azure.account.auth.type." + account_name + ".dfs.core.windows.net", "SharedKey")
        .config("fs.azure.account.key." + account_name + ".dfs.core.windows.net", account_key)
        .getOrCreate()
)

sc = spark.sparkContext

In [3]:
container_name = 'data'
path_to_table = '/masterdata/'

def getDataframeFromAdls(spark_session, container_name, path_to_table, table_name):
    return spark_session.read.parquet(f"abfss://{container_name}@REDACTED.dfs.core.windows.net{path_to_table}{table_name}")

In [4]:
df_qb_file = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qb_file')
        .select(['ID_FILE'])
)

df_qd_state = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qd_state')
        .select(['ID', 'ID_FILE', 'STIME', 'STYPE', 'ACTIVITY_STATE', 'ACTIVITY_CONFID'])
        .withColumnRenamed('ID', 'ID_STATE')
)

df_qd_state_gps = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qd_state_gps')
        .select(['ID_STATE', 'ID_FILE', 'LTIME', 'SOURCE', 'SPEED', 'ACCURACY', 'LON', 'LAT'])
)

df_qd_state_cell = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qd_state_cell')
        .select(['ID_STATE', 'ID_FILE', 'TYPE', 'LEVEL', 'QUAL', 'SLOT'])
)

df_qd_state_wifi = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qd_state_wifi')
        .select(['ID_STATE', 'ID_FILE'])
)

df_qd_state_sense = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qd_state_sens')
        .select(['ID_STATE', 'ID_FILE', 'LIGHT', 'MAGNET_X', 'MAGNET_Y', 'MAGNET_Z', 'PROXIMITY'])
        .withColumn('MAGNET_X', F.when(
            (F.abs(F.col('MAGNET_X')) < 100000), F.col('MAGNET_X'))
            .otherwise(F.lit(None)))
        .withColumn('MAGNET_Y', F.when(
            (F.abs(F.col('MAGNET_X')) < 100000), F.col('MAGNET_X'))
            .otherwise(F.lit(None)))
        .withColumn('MAGNET_Z', F.when(
            (F.abs(F.col('MAGNET_X')) < 100000), F.col('MAGNET_X'))
            .otherwise(F.lit(None)))
)

df_qd_state_batt = (
    getDataframeFromAdls(spark, container_name, path_to_table, 'df_qd_state_batt')
        .select(['ID_STATE', 'ID_FILE', 'BATT_CHARGE'])
)

In [5]:
condition_qdState_qbFile = ['ID_FILE']
condition_qdState_qdStateGps = [
    (df_qd_state.ID_STATE == df_qd_state_gps.ID_STATE) & 
    (df_qd_state.ID_FILE == df_qd_state_gps.ID_FILE) &
    (df_qd_state_gps.LON.between(-180, 180)) & 
    (df_qd_state_gps.LAT.between(-90, 90))
]
condition_qdState_qdStateCell = [
    (df_qd_state.ID_STATE == df_qd_state_cell.ID_STATE) &
    (df_qd_state.ID_FILE == df_qd_state_cell.ID_FILE) &
    (df_qd_state_cell.SLOT == 0)
]
condition_qdState_qdStateWifi = ['ID_FILE', 'ID_STATE']
condition_qdState_qdStateSense = ['ID_FILE', 'ID_STATE']
condition_qdState_qdStateBatt = ['ID_FILE', 'ID_STATE']

In [6]:
df_join = (
    df_qd_state
        .join(F.broadcast(df_qb_file), on=condition_qdState_qbFile, how='inner')
        .join(F.broadcast(df_qd_state_gps), on=condition_qdState_qdStateGps, how='inner')
        .drop(df_qd_state_gps.ID_FILE).drop(df_qd_state_gps.ID_STATE)
        .join(F.broadcast(df_qd_state_cell), on=condition_qdState_qdStateCell, how='left')
        .drop(df_qd_state_cell.ID_FILE).drop(df_qd_state_cell.ID_STATE)
        .join(F.broadcast(df_qd_state_wifi), on=condition_qdState_qdStateWifi, how='left')
        .join(F.broadcast(df_qd_state_sense), on=condition_qdState_qdStateSense, how='left')
        .join(F.broadcast(df_qd_state_batt), on=condition_qdState_qdStateBatt, how='left')
        .cache()
)

In [7]:
'''
df_join_sec = (
    df_join
        .withColumn('STIME', F.col('STIME').cast('string'))
        .withColumn('LTIME', F.col('LTIME').cast('string'))
        .withColumn('STIME_SEC',F.expr('substring(STIME, 1, length(STIME)-3)'))
        .withColumn('LTIME_SEC',F.expr('substring(LTIME, 1, length(STIME)-3)'))
        .withColumn('STIME_SEC', F.col('STIME_SEC').cast('bigint'))
        .withColumn('LTIME_SEC', F.col('LTIME_SEC').cast('bigint'))
        .withColumn('STIME', F.col('STIME').cast('bigint'))
        .withColumn('LTIME', F.col('LTIME').cast('bigint'))
)
'''

"\ndf_join_sec = (\n    df_join\n        .withColumn('STIME', F.col('STIME').cast('string'))\n        .withColumn('LTIME', F.col('LTIME').cast('string'))\n        .withColumn('STIME_SEC',F.expr('substring(STIME, 1, length(STIME)-3)'))\n        .withColumn('LTIME_SEC',F.expr('substring(LTIME, 1, length(STIME)-3)'))\n        .withColumn('STIME_SEC', F.col('STIME_SEC').cast('bigint'))\n        .withColumn('LTIME_SEC', F.col('LTIME_SEC').cast('bigint'))\n        .withColumn('STIME', F.col('STIME').cast('bigint'))\n        .withColumn('LTIME', F.col('LTIME').cast('bigint'))\n)\n"

In [8]:
df_join_sec = (
    df_join
        .withColumn('STIME_SEC', F.round((F.col('STIME') / 1000)).cast(IntegerType()))
        .withColumn('LTIME_SEC', F.round((F.col('LTIME') / 1000)).cast(IntegerType()))
    )

In [9]:
'''(df_join_sec
    .select(['ID_FILE', 'ID_STATE', 'STIME', 'STIME_SEC', 'LTIME_SEC', 'LTIME', 'SPEED'])
    .orderBy(['STIME', 'LTIME'])
    .show(10)
)'''

"(df_join_sec\n    .select(['ID_FILE', 'ID_STATE', 'STIME', 'STIME_SEC', 'LTIME_SEC', 'LTIME', 'SPEED'])\n    .orderBy(['STIME', 'LTIME'])\n    .show(10)\n)"

In [10]:
window_spec  = Window.partitionBy("ID_FILE").orderBy(['ID_FILE', 'ID_STATE'])

In [11]:
def getWindowConditionLagGps(window_spec):
    return (
    (F.col('STIME_SEC') - F.lag(F.col('STIME_SEC')).over(window_spec)).between(0, 900) &
    (F.col('STIME_SEC') - F.col('LTIME_SEC')).between(0, 180) &
    ((F.lag(F.col('STIME_SEC'))).over(window_spec) -
        F.lag(F.col('LTIME_SEC')).over(window_spec))
    .between(0, 180)
    )

In [12]:
def getWindowConditionLeadGps(window_spec):
    return (
    (F.lead(F.col('STIME_SEC')).over(window_spec) - F.col('STIME_SEC')).between(0, 960) &
    (F.col('STIME_SEC') - F.col('LTIME_SEC')).between(0, 180) &
    ((F.lead(F.col('STIME_SEC'))).over(window_spec) -
        F.lead(F.col('LTIME_SEC')).over(window_spec))
    .between(0, 180)
    )

In [13]:
def getWindowConditionLagGpsActivity(window_spec):
    return (
    (F.col('STIME_SEC') - F.lag(F.col('STIME_SEC')).over(window_spec)) < 900
    )

In [14]:
def getWindowConditionLeadGpsActivity(window_spec):
    return (
    (F.lead(F.col('STIME_SEC')).over(window_spec) - F.col('STIME_SEC')) < 900
    )

In [15]:
def getWindowResultGpsSpeed(window_spec, lag_or_lead):
    lagOrLead = lag_or_lead
    return (
    lagOrLead(F.col('SPEED')).over(window_spec)
    )

In [16]:
def getWindowResultGpsActivity(window_spec, lag_or_lead):
    lagOrLead = lag_or_lead
    return (
    lagOrLead(F.col('ACTIVITY_STATE')).over(window_spec)
    )

In [17]:
def getWindowResultLagGpsTime(window_spec):
    return (
    F.col('LTIME_SEC') - F.lag(F.col('LTIME_SEC')).over(window_spec)
    )

In [18]:
def getWindowResultLeadGpsTime(window_spec):
    return (
    F.lead(F.col('LTIME_SEC')).over(window_spec) - F.col('LTIME_SEC')
    )

In [19]:
def getWindowResultLagGpsGeo(window_spec, lat_or_lon):
    return (
    F.lag(F.col(lat_or_lon)).over(window_spec)
    )

In [20]:
def getLagOrLeadGps(df, lag_or_lead, speed_time_activity_geo, window_spec):
    name = f'{lag_or_lead}_GPS_{speed_time_activity_geo}'
    if lag_or_lead == 'LAG':
        condition = getWindowConditionLagGps(window_spec)
        if speed_time_activity_geo == 'TIME':
            result = getWindowResultLagGpsTime(window_spec)
        elif speed_time_activity_geo == 'SPEED':
            result = getWindowResultGpsSpeed(window_spec, F.lag)
        elif speed_time_activity_geo == 'ACTIVITY':
            condition = getWindowConditionLagGpsActivity(window_spec)
            result = getWindowResultGpsActivity(window_spec, F.lag)
        elif speed_time_activity_geo == 'LAT':
            result = getWindowResultLagGpsGeo(window_spec, 'LAT')
        elif speed_time_activity_geo == 'LON':
            result = getWindowResultLagGpsGeo(window_spec, 'LON')
            
    elif lag_or_lead == 'LEAD':
        condition = getWindowConditionLeadGps(window_spec)
        if speed_time_activity_geo == 'TIME':
            result = getWindowResultLeadGpsTime(window_spec)
        elif speed_time_activity_geo == 'SPEED':
            result = getWindowResultGpsSpeed(window_spec, F.lead)
        elif speed_time_activity_geo == 'ACTIVITY':
            condition = getWindowConditionLeadGpsActivity(window_spec)
            result = getWindowResultGpsActivity(window_spec, F.lead)
        
    return (
    df.withColumn(name,
        F.when(condition, result)
            .otherwise(F.lit(None))
        ).cache()
    ) 

In [21]:
df_lag_speed = getLagOrLeadGps(df_join_sec, 'LAG', 'SPEED', window_spec)

In [22]:
df_lead_speed = getLagOrLeadGps(df_lag_speed, 'LEAD', 'SPEED', window_spec)

In [23]:
df_lag_time = getLagOrLeadGps(df_lead_speed, 'LAG', 'TIME', window_spec)

In [24]:
df_lead_time = getLagOrLeadGps(df_lag_time, 'LEAD', 'TIME', window_spec)

In [25]:
df_lag_activity = getLagOrLeadGps(df_lead_time, 'LAG', 'ACTIVITY', window_spec)

In [26]:
df_lead_activity = getLagOrLeadGps(df_lag_activity, 'LEAD', 'ACTIVITY', window_spec)

In [27]:
df_lag_lat = getLagOrLeadGps(df_lead_activity, 'LAG', 'LAT', window_spec)

In [28]:
df_lag_lon = getLagOrLeadGps(df_lag_lat, 'LAG', 'LON', window_spec)

In [29]:
'''
(df_lag_lon
    .select(['STIME_SEC', 'LTIME_SEC', 'SPEED', 'LAG_GPS_SPEED', 'LEAD_GPS_SPEED', 'LAG_GPS_TIME', 'LEAD_GPS_TIME', 'LAG_GPS_ACTIVITY', 'LEAD_GPS_ACTIVITY', 'LAG_GPS_LON', 'LAG_GPS_LAT'])
    .filter((F.col('LAG_GPS_SPEED') > 0) & (F.col('LEAD_GPS_SPEED') > 0))
    .orderBy(['STIME_SEC', 'LTIME_SEC'])
    .show(10))
'''

"\n(df_lag_lon\n    .select(['STIME_SEC', 'LTIME_SEC', 'SPEED', 'LAG_GPS_SPEED', 'LEAD_GPS_SPEED', 'LAG_GPS_TIME', 'LEAD_GPS_TIME', 'LAG_GPS_ACTIVITY', 'LEAD_GPS_ACTIVITY', 'LAG_GPS_LON', 'LAG_GPS_LAT'])\n    .filter((F.col('LAG_GPS_SPEED') > 0) & (F.col('LEAD_GPS_SPEED') > 0))\n    .orderBy(['STIME_SEC', 'LTIME_SEC'])\n    .show(10))\n"

In [30]:
@F.udf(returnType=IntegerType())
def calcDistance(lon_a, lat_a, lon_b, lat_b):
    geo_list = [lon_a,  lat_a, lon_b, lat_b]
    if any(x is None for x in geo_list):
        return None
    else:
        # Transform to radians
        lon_a, lat_a, lon_b, lat_b = map(radians, geo_list)
        dist_lon = lon_b - lon_a
        dist_lat = lat_b - lat_a
        # Calculate area
        area = sin(dist_lat/2)**2 + cos(lat_a) * cos(lat_b) * sin(dist_lon/2)**2
        # Calculate the central angle
        central_angle = 2 * asin(sqrt(area))
        radius = 6371
        # Calculate Distance
        distance = central_angle * radius
        return int(abs(round(distance, 3)) * 1000)

In [31]:
df_distance = (
    df_lag_lon
        .withColumn('DISTANCE', calcDistance(
            F.col('LAG_GPS_LON'), 
            F.col('LAG_GPS_LAT'), 
            F.col('LON'), 
            F.col('LAT')
            )
        )
    )   

In [32]:
'''(df_distance
    .select(['STIME_SEC', 'LTIME_SEC', 'LON', 'LAT', 'LAG_GPS_LON', 'LAG_GPS_LAT', 'DISTANCE'])
    .filter(F.col('DISTANCE') > 0)
    .orderBy(['STIME_SEC', 'LTIME_SEC'])
    .show(50))'''

"(df_distance\n    .select(['STIME_SEC', 'LTIME_SEC', 'LON', 'LAT', 'LAG_GPS_LON', 'LAG_GPS_LAT', 'DISTANCE'])\n    .filter(F.col('DISTANCE') > 0)\n    .orderBy(['STIME_SEC', 'LTIME_SEC'])\n    .show(50))"

In [33]:
df_gps_delay = (
    df_distance
        .withColumn('GPS_DELAY',
            F.when(
                F.col('STIME_SEC') - F.col('LTIME_SEC') < -32000, -32000
            )
            .otherwise(
                F.when(
                    F.col('STIME_SEC') - F.col('LTIME_SEC') > 32000, -32000
                )
                .otherwise(
                    F.col('STIME_SEC') - F.col('LTIME_SEC')
                )
            )
        )
    )

In [34]:
'''(df_gps_delay
    .select(['STIME_SEC', 'LTIME_SEC', 'LON', 'LAT', 'LAG_GPS_LON', 'LAG_GPS_LAT', 'DISTANCE', 'GPS_DELAY'])
    .orderBy(['STIME_SEC', 'LTIME_SEC'])
    .show(50))'''

"(df_gps_delay\n    .select(['STIME_SEC', 'LTIME_SEC', 'LON', 'LAT', 'LAG_GPS_LON', 'LAG_GPS_LAT', 'DISTANCE', 'GPS_DELAY'])\n    .orderBy(['STIME_SEC', 'LTIME_SEC'])\n    .show(50))"

In [35]:
df_pandas = df_gps_delay.toPandas()

In [36]:
import pandas as pd
pd.options.display.max_columns = None
df_pandas['STIME_SEC LTIME_SEC SPEED LAG_GPS_SPEED LEAD_GPS_SPEED LON LAT LAG_GPS_LON LAG_GPS_LAT DISTANCE'.split()].loc[df_pandas['DISTANCE'] > 0].sort_values(['STIME_SEC', 'LTIME_SEC']).head(50)

,STIME_SEC,LTIME_SEC,SPEED,LAG_GPS_SPEED,LEAD_GPS_SPEED,LON,LAT,LAG_GPS_LON,LAG_GPS_LAT,DISTANCE
656437,1588962581,1588962580,0,0.0,0.0,7.015380,49.215580,7.015372,49.215576,1.0
656447,1588972809,1588972809,0,0.0,NaN,7.015378,49.215588,7.015380,49.215580,1.0
656449,1588974538,1588974538,0,0.0,0.0,7.015431,49.215576,7.015378,49.215588,4.0
103976,1589260532,1589260532,10,29.0,10.0,8.037268,50.880848,8.037233,50.881630,87.0
103977,1589260533,1589260532,10,10.0,48.0,8.037258,50.880830,8.037268,50.880848,2.0
103978,1589260563,1589260563,48,10.0,0.0,8.035996,50.879870,8.037258,50.880830,139.0
103979,1589260594,1589260594,0,48.0,1.0,8.032504,50.877907,8.035996,50.879870,328.0
103981,1589261400,1589261400,0,1.0,0.0,8.106959,50.818060,8.032504,50.877903,8462.0
103983,1589261430,1589261414,58,0.0,NaN,8.098655,50.814175,8.106959,50.818060,726.0
103986,1589261490,1589261475,50,49.0,NaN,8.089755,50.807724,8.090233,50.809315,180.0


In [42]:
(df_gps_delay
    .select('STIME_SEC LTIME_SEC SPEED LAG_GPS_SPEED LEAD_GPS_SPEED LON LAT LAG_GPS_LON LAG_GPS_LAT DISTANCE'.split())
    .filter(F.col('DISTANCE') > 0)
    .orderBy(['STIME_SEC', 'LTIME_SEC'])
    .show(50)
)

+----------+----------+-----+-------------+--------------+----------+---------+-----------+-----------+--------+
| STIME_SEC| LTIME_SEC|SPEED|LAG_GPS_SPEED|LEAD_GPS_SPEED|       LON|      LAT|LAG_GPS_LON|LAG_GPS_LAT|DISTANCE|
+----------+----------+-----+-------------+--------------+----------+---------+-----------+-----------+--------+
|1588962581|1588962580|    0|            0|             0| 7.0153804| 49.21558|  7.0153723|  49.215576|       1|
|1588972809|1588972809|    0|            0|          null| 7.0153785|49.215588|  7.0153804|   49.21558|       1|
|1588974538|1588974538|    0|            0|             0| 7.0154314|49.215576|  7.0153785|  49.215588|       4|
|1589260532|1589260532|   10|           29|            10|  8.037268|50.880848|   8.037233|   50.88163|      87|
|1589260533|1589260532|   10|           10|            48|  8.037258| 50.88083|   8.037268|  50.880848|       2|
|1589260563|1589260563|   48|           10|             0|  8.035996| 50.87987|   8.037258|   50

In [45]:
(df_gps_delay
    .select('STIME_SEC LTIME_SEC SPEED LAG_GPS_SPEED LEAD_GPS_SPEED LON LAT LAG_GPS_LON LAG_GPS_LAT     DISTANCE'.split())
    .filter((F.col('LAT') == 50.88163) & 
            (F.col('LON') == 8.037233) |
            (F.col('LAG_GPS_LAT') == 8.037233) |
            (F.col('LAG_GPS_LON') == 8.037233))
    .show()
)

+----------+----------+-----+-------------+--------------+--------+---------+-----------+-----------+--------+
| STIME_SEC| LTIME_SEC|SPEED|LAG_GPS_SPEED|LEAD_GPS_SPEED|     LON|      LAT|LAG_GPS_LON|LAG_GPS_LAT|DISTANCE|
+----------+----------+-----+-------------+--------------+--------+---------+-----------+-----------+--------+
|1589260503|1589260503|   29|         null|            10|8.037233| 50.88163|       null|       null|    null|
|1589260532|1589260532|   10|           29|            10|8.037268|50.880848|   8.037233|   50.88163|      87|
+----------+----------+-----+-------------+--------------+--------+---------+-----------+-----------+--------+



In [ ]:
(df_gps_delay
    .select('STIME_SEC LTIME_SEC SPEED LAG_GPS_SPEED LEAD_GPS_SPEED LON LAT LAG_GPS_LON LAG_GPS_LAT     DISTANCE'.split())
    .filter((F.col('LAT') == 50.88163) & (F.col('LON') == 8.037233))
    .show()
)

In [37]:
'''
df_prev_speed = (
    df_join_sec.withColumn('PREVIOUS_GPS_SPEED',
        F.when(
            (F.col('STIME_SEC') - F.lag(F.col('STIME_SEC')).over(window_spec)).between(0, 900) &
            (F.col('STIME_SEC') - F.col('LTIME_SEC')).between(0, 180) &
            ((F.lag(F.col('STIME_SEC'))).over(window_spec) -
                F.lag(F.col('LTIME_SEC')).over(window_spec))
                .between(0, 180),
            F.lag(F.col('SPEED')).over(window_spec)
        ).otherwise(F.lit(None))).cache())
'''

"\ndf_prev_speed = (\n    df_join_sec.withColumn('PREVIOUS_GPS_SPEED',\n        F.when(\n            (F.col('STIME_SEC') - F.lag(F.col('STIME_SEC')).over(window_spec)).between(0, 900) &\n            (F.col('STIME_SEC') - F.col('LTIME_SEC')).between(0, 180) &\n            ((F.lag(F.col('STIME_SEC'))).over(window_spec) -\n                F.lag(F.col('LTIME_SEC')).over(window_spec))\n                .between(0, 180),\n            F.lag(F.col('SPEED')).over(window_spec)\n        ).otherwise(F.lit(None))).cache())\n"

In [38]:
'''
def getLagOrLeadGps(df, lag_or_lead, speed_time_activity_distance, window_spec):
    name = f'{lag_or_lead}_GPS_{speed_or_time_or_activity}'
    if lag_or_lead == 'LAG':
        condition = getWindowConditionLagGps(window_spec)
        if speed_or_time_or_activity == 'TIME':
            result = getWindowResultLagGpsTime(window_spec)
        elif speed_or_time_or_activity == 'SPEED':
            result = getWindowResultGpsSpeed(window_spec, F.lag)
        elif speed_or_time_or_activity == 'ACTIVITY':
            condition = getWindowConditionLagGpsActivity(window_spec)
            result = getWindowResultGpsActivity(window_spec, F.lag)
            
    elif lag_or_lead == 'LEAD':
        condition = getWindowConditionLeadGps(window_spec)
        if speed_or_time_or_activity == 'TIME':
            result = getWindowResultLeadGpsTime(window_spec)
        elif speed_or_time_or_activity == 'SPEED':
            result = getWindowResultGpsSpeed(window_spec, F.lead)
        elif speed_or_time_or_activity == 'ACTIVITY':
            condition = getWindowConditionLeadGpsActivity(window_spec)
            result = getWindowResultGpsActivity(window_spec, F.lead)
        
    return (
    df.withColumn(name,
        F.when(condition, result)
            .otherwise(F.lit(None))
        ).cache()
    ) 
'''

"\ndef getLagOrLeadGps(df, lag_or_lead, speed_time_activity_distance, window_spec):\n    name = f'{lag_or_lead}_GPS_{speed_or_time_or_activity}'\n    if lag_or_lead == 'LAG':\n        condition = getWindowConditionLagGps(window_spec)\n        if speed_or_time_or_activity == 'TIME':\n            result = getWindowResultLagGpsTime(window_spec)\n        elif speed_or_time_or_activity == 'SPEED':\n            result = getWindowResultGpsSpeed(window_spec, F.lag)\n        elif speed_or_time_or_activity == 'ACTIVITY':\n            condition = getWindowConditionLagGpsActivity(window_spec)\n            result = getWindowResultGpsActivity(window_spec, F.lag)\n            \n    elif lag_or_lead == 'LEAD':\n        condition = getWindowConditionLeadGps(window_spec)\n        if speed_or_time_or_activity == 'TIME':\n            result = getWindowResultLeadGpsTime(window_spec)\n        elif speed_or_time_or_activity == 'SPEED':\n            result = getWindowResultGpsSpeed(window_spec, F.lead)\n     